In [2]:
import re
import csv
from getpass import getpass
from time import sleep
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions 

In [6]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text =  responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet

In [11]:
game = 'lost ark game'#input('search term: ')
keyword_list = [' fun',' interesting', ' stupid',' damn',' bad',' sick',' suck']

for keyword in keyword_list:
    search_term = game+keyword

    # create instance of web driver
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options)

    # navigate to login screen
    driver.get('https://twitter.com/search')
    driver.maximize_window()
    sleep(5)

    # find search input and search for term
    search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
    search_input.send_keys(search_term)
    search_input.send_keys(Keys.RETURN)
    sleep(1)

    # navigate to historical 'latest' tab
    driver.find_element_by_link_text('Latest').click()

    # get all tweets on the page
    data = []
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True

    while scrolling:
        if len(data)>200:
            break
        page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
        for card in page_cards[-15:]:
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    data.append(tweet)

        scroll_attempt = 0
        while True:
            # check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(2) # attempt another scroll
            else:
                last_position = curr_position
                break

    # close the web driver
    driver.close()
    tweets = pd.DataFrame.from_records(data, columns=['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets'])
    tweets.to_excel(search_term+".xlsx", sheet_name="sheet1", index=False)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8628\3924504429.py:10: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)
C:\Users\DELL\AppData\Local\Temp\ipykernel_8628\3924504429.py:10: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)
C:\Users\DELL\AppData\Local\Temp\ipykernel_8628\3924504429.py:10: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-fr

In [7]:
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    if len(data)>200:
        break
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

('The Matsuzakian.', '@Frunzo4000', '2022-09-28T16:34:31.000Z', 'DAMN YOU GOJO! DAMN YOU TO THE FURTHEST REACHES OF NARAKA YOU MOTHERFUCKA. WE NEED THE INVERTED SPEAR OF HEAVEN AND WE NEED THE BLACK ROPE. NOW, MORE THAN EVER. FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK FUCK you stupid motherfucka.\n2', '', '', '2')


KeyboardInterrupt: Interrupted by user

In [37]:
len(data)

53

In [38]:
tweets = pd.DataFrame.from_records(data, columns=['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets'])

In [40]:
tweets.to_excel("naruka_tweets_stupid.xlsx", sheet_name="sheet1", index=False)